Давайте посмотрим на замыкания с интересной стороны. Посмотрим на следующий код:

In [3]:
var createCounter = () => ({
    count: 0,
    increase: function(){
        this.count++
        return this
    },
    decrease: function(){
        this.count--
        return this
    }
})

c = createCounter()
c.increase().increase().decrease()

console.log(c)

{
  count: 1,
  increase: [Function: increase],
  decrease: [Function: decrease]
}


Давайте еще представим что по каким-то причинам нам понадобилось хранить кол-во изменений в счетчике:

In [6]:
var createCounterWithChanges = () => ({
    count: 0,
    changes_count: 0,
    increase: function(){
        this.count++
        this.changes_count++
        return this
    },
    decrease: function(){
        this.count--
        this.changes_count++
        return this
    }
})

c = createCounterWithChanges()
c.increase().increase().decrease()
console.log(c)

{
  count: 1,
  changes_count: 3,
  increase: [Function: increase],
  decrease: [Function: decrease]
}


Сейчас все выглядит вполне адекватно, но вот код который явно приводит к тому, что в счетчике какая-то хрень, а если говорить правильно, то неконсистентные данные:

In [7]:
c = createCounterWithChanges()
c.increase().increase().decrease()
c.count = 100
c.increase()
console.log(c)

{
  count: 101,
  changes_count: 4,
  increase: [Function: increase],
  decrease: [Function: decrease]
}


Как могло так получиться, что в счетчике лежит 101 при 4 изменениях в ту или другую сторону? А если это поведение приводит к какой-то ошибке? Я предлагаю спрятать значение счетчика в замыкание. Сделаем мы это примерно так:

In [11]:
var createClojureCounter = () => {
    var count = 0;
    return {
        increase(){
            count++
            return this
        },
        decrease(){
            count--
            return this
        },
        value(){
            return count
        }
    }
}

Теперь после создания счетчика мы можем менять его значение только через интерфейс предоставленный нам, а само значение будет скрыто от нас в замыкании.

In [13]:
var c  = createClojureCounter()
c.increase().increase().increase()
console.log(c.value())

3
